# Xilinx U280 - WAN experiment on FABRIC

This Jupyter notebook will allow you create a WAN topology for an NEU experiment. 

https://github.com/OCT-FPGA/P4OpenNIC_Public

It is assumed you are operating as part of the FABRIC Maintenance project and have access to the persistent volume named `fpga-tools` created on EDC where releavent tools are downloaded. 

## Step 0: Re-create a VM attached to fpga-tools volume on EDC

In order to have access to necessary tools execute the notebook to [re-create a Storage VM attached](../../fablib_api/fabric_fpgas/fpga_tools_storage.ipynb) to the `fpga-tools` persistent storage. You must execute it as a member of FABRIC Staff project. 

## Step 1: Identify and isolate the worker node

Unless the whole site is already in maintenance, using administrator tools identify the worker node with FPGA and put it in maintenance making sure it does not have experimenter VMs on it. You can check the [aggregate ads in JSON](https://github.com/fabric-testbed/aggregate-ads/tree/main/JSON) to make sure you are targeting the right worker.

## Step 2: Provision a VM on the desired worker with attached FPGA and FABNetv4 connection

Create a slice with a VM attached to the FPGA on the desired site and a FABNetv4 interface to reach the Storage VM in Step 0.

### Initialize fablib and variables

In [ ]:
# Initialize FABlib

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

Define slice parameters - re-execute as needed to run any of the steps in this notebook.

In [ ]:
# user artifact should be deposited Storage VM into /mnt/fpga_tools/static/artifacts/<owner username>/<version> since names of artifacts may be similar or same.
artifact_owner_username = 'sbal'
artifact_version = 'v1'

# edit the name of the user-provided artifact and labtools and DPDK docker images stored in Storage VM as needed
artifact = 'addhdr_jumbo.mcs'

# Xilinx labtools package in Storage VM
labtools_package = 'Vivado/Vivado_Lab_Lin_2023.2_1013_2256.tar.gz'

# Xilinx xbflash2 package in Storage VM
xbflash2_package = 'alveo-packages/xrt_202210.2.13.466_20.04-amd64-xbflash2.deb'

# Xilinx xrt package in Storage VM
xrt_package = 'alveo-packages/xrt_202320.2.16.204_20.04-amd64-xrt.deb'

# Pcimem configDPDK.sh script in Storage VM
configdpdk_script = 'alveo-utilities/configDPDK.sh'

# FABNetv4 of storage VM - consult the Storage VM slice for this FABNetv4 IP address
storage_vm_ip = "10.132.129.2"
# username and password used in storage VM
nginx_user = "fpga_tools"
nginx_password = "secret-password"

#
# should not need to edit below
FPGA_CHOICE='FPGA_Xilinx_U280'


# don't edit - convert from FPGA type to a resource column name
# to use in filter lambda function below
choice_to_column = {
    "FPGA_Xilinx_U280": "fpga_u280_available",
}

column_name = choice_to_column.get(FPGA_CHOICE, "Unknown")

fpga_bdf = "0000:1f:00.0"

#fablib.get_image_names()

In [ ]:
# setup site name
site1='PSC'
node_name1='fpga-node-1'

site2='INDI'
node_name2='fpga-node-2'

# name the slice and the node 
slice_name=f'WAN experiment with {FPGA_CHOICE} on {site1} {site2}'
print(f'Will create slice "{slice_name}"')


### Create a new slice 

Create a slice with FPGA component on selected site and access to FABNetv4 network.

__NOTE:__ It is important to use a Docker-enabled image so that Docker can properly build docker images on IPv6-enabled sites.

In [ ]:
# Create Slice. Note that by default submit() call will poll for 360 seconds every 10-20 seconds
# waiting for slice to come up. Normal expected time is around 2 minutes. 
slice = fablib.new_slice(name=slice_name)
image = 'docker_ubuntu_20'

In [ ]:
# Add node with a 200G drive and 8 of CPU cores using Ubuntu 20 image
node1 = slice.add_node(name=node_name1, site=site1, cores=8, disk=100, image=image)
node1_fpga = node1.add_component(model=FPGA_CHOICE, name='fpga1')
node1_fpga_p1 = node1_fpga.get_interfaces()[0]
node1_fpga_p2 = node1_fpga.get_interfaces()[1]

# be sure to add FABNetv4 so we can communicate with the slice that has the tools
node1.add_fabnet()

# use the postboot script from docker examples
node1.add_post_boot_upload_directory('../../fablib_api/docker_containers/node_tools','.')
node1.add_post_boot_execute('node_tools/enable_docker.sh {{ _self_.image }} ')
node1.add_post_boot_upload_directory('node_config','.')
node1.add_post_boot_execute(f'chmod a+x node_config/ipv6-and-docker-plugins.sh && node_config/ipv6-and-docker-plugins.sh')

In [ ]:
# Add node with a 200G drive and 8 of CPU cores using Ubuntu 20 image
node2 = slice.add_node(name=node_name2, site=site2, cores=8, disk=100, image=image)
node2_fpga = node2.add_component(model=FPGA_CHOICE, name='fpga2')
node2_fpga_p1 = node2_fpga.get_interfaces()[0]
node2_fpga_p2 = node2_fpga.get_interfaces()[1]

# be sure to add FABNetv4 so we can communicate with the slice that has the tools
node2.add_fabnet()

# use the postboot script from docker examples
node2.add_post_boot_upload_directory('../../fablib_api/docker_containers/node_tools','.')
node2.add_post_boot_execute('node_tools/enable_docker.sh {{ _self_.image }} ')
node2.add_post_boot_upload_directory('node_config','.')
node2.add_post_boot_execute(f'chmod a+x node_config/ipv6-and-docker-plugins.sh && node_config/ipv6-and-docker-plugins.sh')

In [ ]:
#nic1_name='SharedNIC1'
#nic2_name='SharedNIC2'
#nic_type='NIC_Basic'
#network1_name='l3-FABNetv4-1'
#network2_name='l3-FABNetv4-2'

#node1_fabnet_iface = node1.add_component(model=nic_type, name=nic1_name).get_interfaces()[0]
#node2_fabnet_iface = node2.add_component(model=nic_type, name=nic2_name).get_interfaces()[0]

## Networks (one in each site)
#net1 = slice.add_l3network(name=network1_name, interfaces=[node1_fabnet_iface], type='IPv4')
#net2 = slice.add_l3network(name=network2_name, interfaces=[node2_fabnet_iface], type='IPv4')

In [ ]:
# Submit Slice Request
slice.submit();

Add storage VM into /etc/hosts for convenience. __Consult the storage slice for the FABNetv4 IPv4 address of that VM.__

In [ ]:
slice = fablib.get_slice(slice_name)

for node in slice.get_nodes():
   node_name = node.get_name()
   print(f"{node_name}")

   commands = list()
   commands.append(f"echo {storage_vm_ip} fpga-tools-host | sudo tee -a /etc/hosts")
   commands.append(f"echo 127.0.0.1 {node_name} | sudo tee -a /etc/hosts")

   for command in commands:
      stdout, stderr = node.execute(command)

## Step 3: Inspect the slice
Note that nat64 configuration is done at boot time.

In [ ]:
slice = fablib.get_slice(slice_name)

for node in slice.get_nodes():
   node_name = node.get_name()
   node_addr = node.get_interface(network_name=f'FABNET_IPv4_{node.get_site()}').get_ip_addr()

   print(f"{node_name}")
   print(f'Node FABNetV4 IP Address is {node_addr}')

slice.show()
slice.list_nodes()
slice.list_networks()

## Step 4: Modify slice
Add FPGAs

In [ ]:
#slice = fablib.get_slice(slice_name)

#node1 = slice.get_node(name=node_name1)              
#node2 = slice.get_node(name=node_name2)              

#print(f"{node1.get_interfaces()[0]}")
#print(f"{node1.get_interfaces()[1]}")
#print(f"{node1.get_interfaces()[2]}")

#print(f"{node2.get_interfaces()[0]}")
#print(f"{node2.get_interfaces()[1]}")
#print(f"{node2.get_interfaces()[2]}")

#fpga_n1_p1 = node1.get_interfaces()[0]
#fpga_n2_p1 = node2.get_interfaces()[0]

print(f"--- node1_fpga_p1:\n{node1_fpga_p1}")
print(f"--- node1_fpga_p2:\n{node1_fpga_p2}")

print(f"--- node2_fpga_p1:\n{node2_fpga_p1}")
print(f"--- node2_fpga_p2:\n{node2_fpga_p2}")



In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

network1_name='net1'

port1 = node1_fpga_p2
port2 = node2_fpga_p1

# Use L2Bridge network services to connect the smart NIC and the FPGA ports
net1 = slice.add_l2network(name=network1_name, interfaces=[port1, port2], subnet=IPv4Network("192.168.40.0/24"), type='L2STS')

###net1 = slice.add_l2network(name=network1_name, subnet=IPv4Network("192.168.40.0/24"), type='L2STS')
###net1.add_interface(node1_fpga_p1)
###net1.add_interface(node2_fpga_p1)


In [ ]:
slice.submit()

# Slice Info

## Slice

In [ ]:
slice = fablib.get_slice(name=slice_name)
slice.show();

## Nodes and Components

In [ ]:
node = slice.get_node(node_name1) 
node.show()

fpga = node.get_component('fpga1')
fpga.show();

print(f"--- fpga_p1:\n{fpga.get_interfaces()[0]}")
print(f"--- fpga_p2:\n{fpga.get_interfaces()[1]}")

In [ ]:
node = slice.get_node(node_name2) 
node.show()

fpga = node.get_component('fpga2')
fpga.show();

print(f"--- fpga_p1:\n{fpga.get_interfaces()[0]}")
print(f"--- fpga_p2:\n{fpga.get_interfaces()[1]}")

## Step 5: Fetch Tools

Fetch the Linux tools from Xilinx and place into appropriate location

Clone OCT-FPGA repo

In [ ]:
# checkout the repo
tools_neu_repo = 'https://github.com/OCT-FPGA/P4OpenNIC_Public.git'

commands = list()
commands.append(f'[ ! -d ~/fpga-tools ] && mkdir -p ~/fpga-tools')
commands.append(f'cd fpga-tools && git clone {tools_neu_repo}')

for node in slice.get_nodes():
   node_name = node.get_name()
   for command in commands:
      print(f'--- Node {node_name}: Executing command: {command}')
      stdout, stderr = node.execute(command)
   print('--- Done')

Clone pcimem repo

In [ ]:
# checkout the repo - pcimem
pcimem_repo = 'https://github.com/billfarrow/pcimem.git'

commands = list()
commands.append(f'[ ! -d ~/fpga-tools ] && mkdir -p fpga-tools')
commands.append(f'cd fpga-tools && git clone {pcimem_repo}')

for node in slice.get_nodes():
   node_name = node.get_name()
   for command in commands:
      print(f'--- Node {node_name}: Executing command: {command}')
      stdout, stderr = node.execute(command)
   print('--- Done')

Copy configDPDK.sh from the Storage VM

In [ ]:
import os.path

tools_location = '~/fpga-tools'

commands = list()
commands.append(f'[ ! -d {tools_location} ] && mkdir -p {tools_location}')
commands.append(f'curl -k -u {nginx_user}:{nginx_password} https://fpga-tools-host/fpga-tools/{configdpdk_script}  > {tools_location}/{os.path.basename(configdpdk_script)}')
commands.append(f'chmod +x {tools_location}/{os.path.basename(configdpdk_script)}')

for node in slice.get_nodes():
   node_name = node.get_name()
   for command in commands:
      print(f'--- Node {node_name}: Executing command: {command}')
      stdout, stderr = node.execute(command)
   print('--- Done')

Clone open-nic-driver repo

In [ ]:
# checkout the repo - open-nic-driver
opennicdriver_repo = 'https://github.com/Xilinx/open-nic-driver.git'

commands = list()
commands.append(f'[ ! -d ~/fpga-tools ] && mkdir -p fpga-tools')
commands.append(f'cd ~/fpga-tools && git clone {opennicdriver_repo}')
commands.append(f'cd ~/fpga-tools/open-nic-driver && git checkout 2fa9668')

for node in slice.get_nodes():
   node_name = node.get_name()
   for command in commands:
      print(f'--- Node {node_name}: Executing command: {command}')
      stdout, stderr = node.execute(command)
   print('--- Done')

Update packages

In [ ]:
commands = list()
commands.append(f'(sudo apt -y update && sudo apt -y upgrade) &> /tmp/fpga-apt-upgrade.log')

for node in slice.get_nodes():
   node_name = node.get_name()
   for command in commands:
      print(f'--- Node {node_name}: Executing command: {command}')
      stdout, stderr = node.execute(command)
   print('--- Done')

Reboot

In [ ]:
reboot = 'sudo reboot'

try:
   for node in slice.get_nodes():
      node_name = node.get_name()
      print(f'--- Node {node_name}: reboot')
      node.execute(reboot)
      slice.wait_ssh(timeout=360,interval=10,progress=True)

      print("Now testing SSH abilites to reconnect...",end="")
      slice.update()
      slice.test_ssh()
      print("Reconnected!")
       
except Exception as e:
    print(f"Fail: {e}")  

for node in slice.get_nodes():
   node_name = node.get_name()
   print(f'--- Node {node_name}: node.config')
   node.config()
   print("--- Done")  

Install tools (add various package installations to this section)

In [ ]:
commands = list()
commands.append(f'sudo apt install -y hping3 &> /tmp/fpga-apt-tools.log')

for node in slice.get_nodes():
   node_name = node.get_name()
   for command in commands:
      print(f'--- Node {node_name}: Executing command: {command}')
      stdout, stderr = node.execute(command)
   print('--- Done')

Install kernel-headers

In [ ]:
commands = list()
commands.append(f'sudo apt install -y linux-headers-$(uname -r) &> /tmp/fpga-apt-kernel-headers.log')

for node in slice.get_nodes():
   node_name = node.get_name()
   for command in commands:
      print(f'--- Node {node_name}: Executing command: {command}')
      stdout, stderr = node.execute(command)
   print('--- Done')

## Step 6: Install Vivado Lab

Install Vivado Lab Tools

Next transfer the Xilinx Vivado Lab package from the Storage VM

In [ ]:
import os.path

tools_location = '~/xilinx-labtools/vivado-installer'

commands = [f'[ ! -d {tools_location} ] && mkdir -p {tools_location}',
            f'curl -k -u {nginx_user}:{nginx_password} https://fpga-tools-host/fpga-tools/{labtools_package}  > {tools_location}/{os.path.basename(labtools_package)}']

for node in slice.get_nodes():
   node_name = node.get_name()
   for command in commands:
      print(f'--- Node {node_name}: Executing command: {command}')
      stdout, stderr = node.execute(command)
   print('--- Done')

Next transfer the artifact from Storage VM into a folder in this repo. This file will be called `<application_name>.mcs` and should be placed in the `~/bitfile` directory before starting the firmware build.

In [ ]:
bitfile_location = '~/fpga-bitfile'

commands = [f'[ ! -d {bitfile_location} ] && mkdir -p {bitfile_location}',
            f'curl -k -u {nginx_user}:{nginx_password} https://fpga-tools-host/fpga-tools/artifacts/{artifact_owner_username}/{artifact_version}/{artifact}  > {bitfile_location}/{artifact}']


for node in slice.get_nodes():
   node_name = node.get_name()
   for command in commands:
      print(f'--- Node {node_name}: Executing command: {command}')
      stdout, stderr = node.execute(command)
   print('--- Done')

Extract and install Xilinx Vivado Lab package

In [ ]:
# parse package name to get version
import re

pattern = re.compile('(Xilinx_)?Vivado_Lab_Lin_([\d]{4}.[\d]_[\d]{4}_[\d]{4}).tar.gz')
m = pattern.match(os.path.basename(labtools_package))
if m:
    version = m[2]
else:
    version = 'Unknown'


vivado_install_dir = "/tools/Xilinx"
 
commands = list()
commands.append(f'tar xf {tools_location}/{os.path.basename(labtools_package)} -C {tools_location}')
commands.append(f'sudo {tools_location}/Vivado_Lab_Lin_{version}/xsetup --agree 3rdPartyEULA,XilinxEULA --batch Install --edition "Vivado Lab Edition (Standalone)" --location {vivado_install_dir}')


for node in slice.get_nodes():
   node_name = node.get_name()
   for command in commands:
      print(f'--- Node {node_name}: Executing command: {command}')
      stdout, stderr = node.execute(command)
   print('--- Done')

Install necessary libs

In [ ]:
commands = list()
commands.append(f'sudo apt install -y lsb &> /tmp/fpga-apt-install.out')
commands.append(f'sudo {tools_location}/Vivado_Lab_Lin_{version}/installLibs.sh &> /tmp/fpga-installLibs.out')

for node in slice.get_nodes():
   node_name = node.get_name()
   for command in commands:
      print(f'--- Node {node_name}: Executing command: {command}')
      stdout, stderr = node.execute(command)
   print('--- Done')

Install JTAG drivers

In [ ]:
import re

pattern = re.compile('([\d]{4}.[\d])_([\d]{4}_[\d]{4})')
m = pattern.match(os.path.basename(version))
if m:
    version_major = m[1]
else:
    version_major = 'Unknown'


commands = list()
commands.append(f'sudo {vivado_install_dir}/Vivado_Lab/{version_major}/data/xicom/cable_drivers/lin64/install_script/install_drivers/install_drivers')

for node in slice.get_nodes():
   node_name = node.get_name()
   for command in commands:
      print(f'--- Node {node_name}: Executing command: {command}')
      stdout, stderr = node.execute(command)
   print('--- Done')

Update packages

In [ ]:
commands = list()
commands.append(f'(sudo apt -y update && sudo apt -y upgrade) &> /tmp/fpga-apt-upgrade.log')

for node in slice.get_nodes():
   node_name = node.get_name()
   for command in commands:
      print(f'--- Node {node_name}: Executing command: {command}')
      stdout, stderr = node.execute(command)
   print('--- Done')

#### Reboot

In [ ]:
reboot = 'sudo reboot'

try:
   for node in slice.get_nodes():
      node_name = node.get_name()
      print(f'--- Node {node_name}: reboot')
      node.execute(reboot)
      slice.wait_ssh(timeout=360,interval=10,progress=True)

      print("Now testing SSH abilites to reconnect...",end="")
      slice.update()
      slice.test_ssh()
      print("Reconnected!")
       
except Exception as e:
    print(f"Fail: {e}")  

for node in slice.get_nodes():
   node_name = node.get_name()
   print(f'--- Node {node_name}: node.config')
   node.config()
   print("--- Done")  

#### Get the JTAG ID


In [ ]:
commands = list()
commands.append(f'source {vivado_install_dir}/Vivado_Lab/{version_major}/settings64.sh && vivado_lab -mode batch -source ~/fpga-tools/P4OpenNIC_Public/FABRIC-P4/scripts/get_jtag.tcl | grep -o "Xilinx/[^[:space:]]*" | cut -d/ -f2 > /tmp/fpga-jtag.id')
commands.append(f'echo {fpga_bdf} > /tmp/fpga-bdf.id')

for node in slice.get_nodes():
   node_name = node.get_name()
   for command in commands:
      print(f'--- Node {node_name}: Executing command: {command}')
      stdout, stderr = node.execute(command)
   print('--- Done')

Install kernel-headers

In [ ]:
commands = list()
commands.append(f'sudo apt install -y linux-headers-$(uname -r) &> /tmp/fpga-apt-kernel-headers.log')

for node in slice.get_nodes():
   node_name = node.get_name()
   for command in commands:
      print(f'--- Node {node_name}: Executing command: {command}')
      stdout, stderr = node.execute(command)
   print('--- Done')

Install libraries for running Vivado GUI

In [ ]:
commands = list()
commands.append(f'sudo apt-get install -y libxrender1 libxtst6 libxi6 &> /tmp/fpga-apt-libs.log')

for node in slice.get_nodes():
   node_name = node.get_name()
   for command in commands:
      print(f'--- Node {node_name}: Executing command: {command}')
      stdout, stderr = node.execute(command)
   print('--- Done')

### Install Alveo Packages
https://www.xilinx.com/support/download/index.html/content/xilinx/en/downloadNav/alveo/u280.html

#### Step 1. Download the Xilinx Runtime
 
The Xilinx runtime (XRT) is a low level communication layer (APIs and drivers) between the host and the card.
- [xrt_202320.2.16.204_20.04-amd64-xrt.deb](https://www.xilinx.com/bin/public/openDownload?filename=xrt_202320.2.16.204_20.04-amd64-xrt.deb)
 
#### Step 2. Download the Deployment Target Platform

The deployment target platform is the communication layer physically implemented and flashed into the card.
- [xilinx-u280-gen3x16-xdma_2023.2_2023_1014_0238-all.deb.tar.gz](https://www.xilinx.com/bin/public/openDownload?filename=xilinx-u280-gen3x16-xdma_2023.2_2023_1014_0238-all.deb.tar.gz)


In [ ]:
#commands = list()
#commands.append(f'sudo wget -qO - https://www.xilinx.com/support/download/2020-2/xilinx-master-signing-key.asc | sudo apt-key add -')
#commands.append(f'echo "deb https://packages.xilinx.com/artifactory/debian-packages $(lsb_release -cs) main" | sudo tee -a /etc/apt/sources.list.d/xlnx.list')
#commands.append(f'(sudo apt-get update && sudo apt-get install -y xilinx-u280-gen3x16-xdma-base=1-3585717 xilinx-u280-gen3x16-xdma-validate=1-3585755 xilinx-cmc-u280=1.3.5-3592445 xilinx-sc-fw-u280=4.3.28-1.ea1b92f ) &> /tmp/fpga-apt-alveo.log')


#for node in slice.get_nodes():
#   node_name = node.get_name()
#   for command in commands:
#      print(f'--- Node {node_name}: Executing command: {command}')
#      stdout, stderr = node.execute(command)
#   print('--- Done')

Install xbflash2

In [ ]:
import os.path

tools_location = '~/xilinx-labtools/alveo-packages'

commands = list()
commands.append(f'[ ! -d {tools_location} ] && mkdir -p {tools_location}')
commands.append(f'wget -q https://www.xilinx.com/bin/public/openDownload?filename=xrt_202210.2.13.466_20.04-amd64-xbflash2.deb -O {tools_location}/xrt_202210.2.13.466_20.04-amd64-xbflash2.deb')
commands.append(f'curl -k -u {nginx_user}:{nginx_password} https://fpga-tools-host/fpga-tools/{xbflash2_package}  > {tools_location}/{os.path.basename(xbflash2_package)}')
commands.append(f'sudo apt-get install {tools_location}/xrt_202210.2.13.466_20.04-amd64-xbflash2.deb &> /tmp/fpga-apt-xbflash2.log')

for node in slice.get_nodes():
   node_name = node.get_name()
   for command in commands:
      print(f'--- Node {node_name}: Executing command: {command}')
      stdout, stderr = node.execute(command)
   print('--- Done')

## Step 7: Kernel modules

#### Step 1. Use configDPDK to read the registers

In [ ]:
tools_location = '~/fpga-tools'

commands = list()
commands.append(f'cd {tools_location}/pcimem; make; [ ! -e {tools_location}/pcimem/configDPDK.sh ] && ln -s {tools_location}/configDPDK.sh')
commands.append(f'cd {tools_location}/pcimem && ./configDPDK.sh')

for node in slice.get_nodes():
   node_name = node.get_name()
   for command in commands:
      print(f'--- Node {node_name}: Executing command: {command}')
      stdout, stderr = node.execute(command)
   print('--- Done')

#### Step 2. Install kernel module open-nic-driver

In [ ]:
tools_location = '~/fpga-tools'

commands = list()
commands.append(f'cd {tools_location}/open-nic-driver; make')
commands.append(f'sudo rmmod onic &> /dev/null')
commands.append(f'cd {tools_location}/open-nic-driver; sudo insmod onic.ko RS_FEC_ENABLED=0')
commands.append(f'lsmod | grep onic')

for node in slice.get_nodes():
   node_name = node.get_name()
   for command in commands:
      print(f'--- Node {node_name}: Executing command: {command}')
      stdout, stderr = node.execute(command)
   print('--- Done')

In [ ]:
#### Step 3. Configure interfaces

In [ ]:
fpga_interface = 'enp31s0f0'
commands = list()
commands.append(f'sudo ip link set {fpga_interface} up')

for node in slice.get_nodes():
   node_name = node.get_name()
   for command in commands:
      print(f'--- Node {node_name}: Executing command: {command}')
      stdout, stderr = node.execute(command)
   print('--- Done')


node1 = slice.get_node(name=node_name1)        
node2 = slice.get_node(name=node_name2)        
node1_addr = '192.168.40.11'
node2_addr = '192.168.40.12'

stdout, stderr = node1.execute(f'sudo ip addr add {node1_addr}/24 dev {fpga_interface}')
stdout, stderr = node2.execute(f'sudo ip addr add {node2_addr}/24 dev {fpga_interface}')

stdout, stderr = node1.execute(f'ping -c 30 {node2_addr}')


## Step 8: Extend Slice

Get slice details and extend the slice. This cell is optional and can be executed as-needed.

In [ ]:
slice = fablib.get_slice(name=slice_name)
slice.show();

Renew by 14 days

In [ ]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

# Set end host to now plus 14 days
end_date = (datetime.now(timezone.utc) + timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fablib.get_slice(name=slice_name)

    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")

## Step 9: Delete the slice

Delete the slice after completing the programming.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")